In [11]:
%load_ext autoreload
%autoreload 2

import os
from tqdm.notebook import tqdm
import dask.array as da

import pandas as pd
import numpy as np
import pickle as pkl
import napari
import seaborn as sns
import matplotlib.pyplot as plt
from tifffile import imread

from sem_quant.load_config import load_config
from sem_quant.utils import smart_path
from sem_quant.processing_utils import mask_from_df

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
config_file_name = "../configs/A01_config.json"

In [10]:
# read in config file

config = load_config(config_file_name)

im_path = smart_path(config.paths.im_path)
analysis_dir = smart_path(config.paths.analysis_dir)

output_prefix = config.paths.output_prefix
mitos_data_suffix = config.paths.mitos_data_suffix

px_size = config.data_properties.px_size
axons_res = config.data_properties.axons_res
mitos_res = config.data_properties.mitos_res
row_offset = config.data_properties.row_offset
col_offset = config.data_properties.col_offset

# details of selecting mito masks
res_adjust = axons_res - mitos_res

df_axons_path = os.path.join(analysis_dir,f'{output_prefix}axons.pkl')
df_mitos_path = os.path.join(analysis_dir,f'{output_prefix}mitos.pkl')


## Get an image

In [12]:
store = imread(im_path, aszarr=True)
im_list = []
for res in range(axons_res+1):
    im = da.from_zarr(store,res)
    im_list.append(im)

In [13]:
im_shape = im_list[0].shape
im_shape

(51069, 64145)

In [14]:
viewer = napari.Viewer()
viewer.add_image(im_list, scale = [px_size, px_size])

<Image layer 'im_list' at 0x1cb94e1ee10>

In [15]:
viewer.scale_bar.visible = True

# Text options
viewer.scale_bar.unit = 'nm'  # set to None to diplay no unit
viewer.scale_bar.length = 5000  # length, in units, of the scale bar
viewer.scale_bar.font_size = 20  # default is 10

# Text color
viewer.scale_bar.colored = True  # default value is False
viewer.scale_bar.color = 'white'  # default value is magenta: (1,0,1,1)

# Background box
viewer.scale_bar.box = False  # add background box, default is False

# Scale bar position
viewer.scale_bar.position = 'bottom_right'  # default is 'bottom_right'
viewer.scale_bar.ticks = False

## Visualize axons

In [16]:
df_name = f'{output_prefix}axons.pkl'
df_path = os.path.join(analysis_dir,df_name)
df_cells = pd.read_pickle(df_path)
print('Number of axons:',len(df_cells))

Number of axons: 1308


In [17]:
im_shape = im_list[axons_res-mitos_res].shape

# add mask to viewer
mask = mask_from_df(df_cells, im_shape)
viewer.add_labels(mask, scale = [px_size*2**axons_res,px_size*2**axons_res])

<Labels layer 'mask' at 0x1cba4e17c50>

## Visualize mitochondria

In [18]:
df_name = f'{output_prefix}mitos.pkl'
df_path = os.path.join(analysis_dir,df_name)
df_mitos = pd.read_pickle(df_path)
print('Number of axons:',len(df_mitos))

Number of axons: 732


In [19]:
# add points for each category
categories = ['healthy', 'swollen']
colors = ['green', 'red']

for cat, color in zip(categories, colors):

    points = df_mitos.loc[df_mitos.prediction == cat,['centroid-0','centroid-1']].to_numpy()
    
    viewer.add_points(points, name = cat, size = 50, border_color= color, face_color = [0, 0, 0, 0], scale = [px_size*2**mitos_res,px_size*2**mitos_res])